## ADP - Neural Network Value Function

In this Notebook we implemented the ADP algorithm using neural network as value function. Distributions of demand in each location is either normal or poisson.

Code is documented with comments and markdown blocks.

In [1]:
# dependencies
import random
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import *
import tensorflow as tf
from tensorflow import keras

In [2]:
# sets
locations = 2
periods = 10

# initial state
initialInventory = [70, 70] # initial inventory
total_bikes = sum(initialInventory)

# extrogenous info
demandMean = [50, 90] 

In [4]:
def poisson(mean, lambd):
    return round(mean + np.random.poisson(lambd, 1)[0])

def normal(mean, std):
    return round(np.random.normal(mean, std, 1)[0])

### Optimal Action Linear Program

The following function defines a Linear Program for finding the best action in the problem. Action space is very high: shifting one bike from location i to j, shifting two bikes from location j to k etc... All of those combinations can be seen as different actions, depending also on the inventory levels.

In [5]:
"""
locations: number of locations
holding_cost: array containing the holding cost per unit in each location
trans_cost: matrix with the cost for moving one unit from a location to another one
inventory: array with current state (number of bikes per location), each value represent one location
total_bikes: total number of bikes in the network
shiftable_bikes: array containing the number of bikes that can be shifted (if positive) or taken (if negative) from/in a location
env: gurobipy configuration
"""
def best_action(locations, holding_cost, penalty_cost, trans_cost, inventory, total_bikes, shiftable_bikes, env=None):
    demands = []
    for i in range(locations):
        demands.append(inventory[i] - shiftable_bikes[i])
    print("demands"+str(demands))
    print("inventory"+str(inventory))

    lp = gp.Model("Transshipment Decision Linear Programming", env=env)
    y = {} # number of bikes at each location after transshipment inventory
    z = {} # transshipment decision
    l = {} # linearize penalty_cost
    h = {} # linearize holding_cost
    for i in range(locations):
        y[i] = lp.addVar(lb=0, vtype=GRB.INTEGER)
        l[i] = lp.addVar()
        h[i] = lp.addVar()
    for i in range(locations):
        for j in range(locations):
            z[i,j] = lp.addVar(lb=0, vtype=GRB.INTEGER)
    
    # preserve costant number of bikes if we sum all locations # capacity constraint
    lp.addConstr(sum(y[i] for i in range(locations)) == total_bikes)

    # transshipment decision
    for i in range(locations):
        lp.addConstr(y[i] == inventory[i] + sum(z[j,i] - z[i,j] for j in range(locations)))

    # transshipment amount=inventory at retailer i at time t
    for i in range(locations):
        lp.addConstr(sum(z[i,j] for j in range(locations)) == inventory[i])
    
    # linearize penalty_cost: max(0, d[i]-y[i])
    for i in range(locations):
        lp.addConstr(l[i] >= 0)
        lp.addConstr(l[i] >= float(demands[i]) - y[i])
    
    # linearize holding_cost: max(0, y[i]-d[i])
    for i in range(locations):
        lp.addConstr(h[i] >= 0)
        lp.addConstr(h[i] >= y[i] - demands[i])
     
    obj = sum(trans_cost[i][j]*z[i,j] for i in range(locations) for j in range(locations)) + sum(penalty_cost[i]*l[i] + holding_cost[i]*h[i] for i in range(locations))
    lp.setObjective(obj, GRB.MINIMIZE)
    lp.optimize()
    
    solution_z = lp.getAttr('x', z)
    for i in range(locations):
        for j in range(locations):
            if solution_z[i,j] > 0:
                print ('%s -> %s ship: %g' % (i,j,solution_z[i,j]))

    z = []
    for i in range(locations):
        z.append([])
        for j in range(locations):
            z[i].append(solution_z[i,j])

    return lp.objVal, z

### Neural Network Value Function

Value function is this case is represented by a Neural Network model. Hence, data is encoded into a weight and a bias. Input is the demand and it tries to predict how many bikes can be shifted from each location. We initialize one model for each location so that it can be easier handling different distributions of the demand.

Technically speaking, we train the weights with the well known Mean Least Squares approach and Stochastic Gradient Descent using the Tensorflow library.

The main function in the following block represents the update of the weights that is called at each iteration of the ADP algorithm.

We used different regularizing techniques, such a L2 regularizer, a dropout layer, to avoid overfittting, paritcularly at the initial periods where we have litte data to train. Also, we used learning rates and momentums to help escape from local minimum and find global minimum.

In [7]:
def nn(available_bikes, demand, inventory_nr, locations): #classifier
    train_X, train_Y = [], []
    for i in range(len(available_bikes) - 1):
        train_X.append([available_bikes[i][inventory_nr]])
        train_Y.append([demand[i+1][inventory_nr] - available_bikes[i][inventory_nr]]) # shiftable bikes

    train_X = np.array(train_X)
    train_Y = np.array(train_Y)
    
    model = keras.Sequential([
    keras.layers.Dense(units = locations^3, 
                       activation='relu',
                       kernel_regularizer=keras.regularizers.l2(0.1),
                       input_shape=(train_X.shape[-1],)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1, activation='linear')])

    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=0.0001, momentum=0.3),
        loss=keras.losses.MeanSquaredError(),
        metrics=[keras.metrics.MeanSquaredError(name='mean_squared_error')])
    
    history = model.fit(x=train_X, 
                        y=train_Y,
                        epochs=5, 
                        batch_size=10,
                        shuffle = True,
                        verbose=0)
    
    MSE = history.history['mean_squared_error'][-1]
    
    return model, MSE

def init_linear_model():
    weights = np.random.randn(1, 1) / 4
    bias = np.random.randn(1, 1) / 4

    return [weights, bias]


"""
available_bikes: number of bikes at a specific location
"""
def predict_shiftable_demand(weights, bias, available_bikes):
    return weights * available_bikes + bias

### Approximate Dynamic Programming

As seen during the lecture, the main logic of ADP is contained into the main loop which has three phases:
- Make a decision
- Update the value function
- Determine next observation state

This is exactly what the snippet below is doing. For the action part, we rely on the Linear Program seen before. The decision is always taken: this should be interpreted as if we have a transition probability equals to 1 for the chosen action. In our bike sharing scenario we can translate it with "bikes are always shifted according to the plan" (trucks never break etc...). In the Linear Program we also input the output of the value function.

Value function is updated accordinly to the model we use. For moving average, we can just add the new observation to a vector and last N values can be used for calculating the value. For linear regression and neural network, we use stochastic gradient descent. That way, we can handle training (read updating) in an online fashion easily.

For updating the observation space for the next iteration, we just need to apply transshipment decision retrieved from the Linear Program. 

As far as the running average is concerned, the implementation simply keeps each observation in a specific array and computes the mean when needed.

In [8]:
def ADP(locations, periods, holding_cost, penalty_cost, trans_cost, initialInventory, demand_dist, demandMean, demandPara, total_bikes):
    # initialize post-decison value function approximation
    costs = []      
    # initialize after transshipment inventory per location per period
    y = []
    # initialize transshipment decision per location per period
    z = [[[None]*locations for _ in range(locations)] for _ in range(periods)] 
    # before transshipment inventory x_it at x_i0 = S_i
    inventory = []
    inventory.append(initialInventory)
    # initialize demands
    demands = []
        
    linear_models = []
    for _ in range(locations):
        linear_models.append(init_linear_model())
        
    ls_err_history = []
        
    for t in range(periods):
        # 1. calculate new demands for current day
        if demand_dist == "normal":
            demands.append([normal(demandMean[i], demandPara[i]) for i in range(locations)])
        elif demand_dist == "poisson":
            demands.append([poisson(demandMean[i], demandPara[i]) for i in range(locations)])
            
        # 2. make a decision - predict shiftable + linear model for best shift of bikes + calculate cost
        if t <= 1: # simple linear regression used for the first two iterations until nn model is built in the 3rd period
            shiftable_bikes = []
            for i in range(locations):
                shiftable_bikes.append(predict_shiftable_demand(linear_models[i][0], linear_models[i][1], inventory[t][i]))
        else: # neural network 
            shiftable_bikes = []
            for i in range(locations):
                shiftable_bikes.append(linear_models[i].predict([inventory[t][i]]))
            
        print("shiftable_bikes"+str(shiftable_bikes))
        
        # linear program: calcualte cost and decision
        value, z_t = best_action(locations, holding_cost, penalty_cost, trans_cost, inventory[t], total_bikes, shiftable_bikes, env=None)
        costs.append(value)
        for i in range(locations):
            for j in range(locations):
                z[t][i][j] = z_t[i][j]
            
        # 3. update value function approximation
        errs = []
        if t > 0:
            for i in range(locations):
                estimator, err = nn(inventory, demands, i, locations)
                linear_models[i] = estimator
                errs.append(err)
            ls_err_history.append(err) 
            
        # 4. make transshipment decision to determine the next state
        new_inventory = [inventory[t][i] + sum(z[t][j][i] -z[t][i][j] for j in range(locations)) for i in range(locations)]
        inventory.append(new_inventory)
                
    return sum(costs)/periods

### Performances
The table below represents the average cost per period, which varies in different setting of cost and demand parameters. We expect to see the model can help predict cost consistently even if the variation of demand becomes larger. However, the model fails to do so.

In [9]:
# cost parameters
holding_cost = [[2, 2]] # per location i
penalty_cost = [[2, 2], [4, 4]] # per location i
trans_cost = [[[0, 1],[1, 0]],[[0, 1],[3, 0]]] # from i to j

# demand parameters
std_demands = [[5, 5], [10, 10]]
delta_demands = [[1, 1], [2, 2]]
demand_dist = {"normal":std_demands, "poisson":delta_demands}

perform_dict = {}
n=0
for holdingcost in holding_cost:
    for penaltycost in penalty_cost:
        for transcost in trans_cost:
            for demanddist, i in demand_dist.items():
                for demandpara in i:
                    cost = ADP(locations,periods,holdingcost,penaltycost,transcost,initialInventory,demanddist,demandMean,demandpara,total_bikes)
                    perform_dict[n]=[penaltycost[0],transcost,demanddist,demandpara[0],cost]
                    n+=1

summary_performance = pd.DataFrame.from_dict(perform_dict, orient = 'index', columns = ['penalty cost','transportation cost', 'demand distribution', 'demand parameters', 'Objective'])
summary_performance

shiftable_bikes[array([[1.85308842]]), array([[3.90899345]])]
demands[array([[68.14691158]]), array([[66.09100655]])]
inventory[70, 70]
Academic license - for non-commercial use only - expires 2021-09-05
Using license file /Users/mingyutu/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 10 columns and 24 nonzeros
Model fingerprint: 0x2ed53e4e
Variable types: 4 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+02]
Presolve removed 12 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros
Variable types: 3 continuous, 2 integer (0 binary)

Root relaxation: objective 1.152416e+01, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept


Explored 1 nodes (1 simplex iterations) in 0.11 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 42.4715 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.247150421143e+01, best bound 4.247150421143e+01, gap 0.0000%
0 -> 0 ship: 70
1 -> 1 ship: 70
shiftable_bikes[array([[-0.05351914]], dtype=float32), array([[0.03804676]], dtype=float32)]
demands[array([[70.05352]], dtype=float32), array([[69.96195]], dtype=float32)]
inventory[70.0, 70.0]
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 10 columns and 24 nonzeros
Model fingerprint: 0x5db7b1d0
Variable types: 4 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+02]
Presolve removed 12 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 5 columns


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   43.98760    0    1          -   43.98760      -     -    0s
H    0     0                     117.1393252   43.98760  62.4%     -    0s
H    0     0                      45.1393252   43.98760  2.55%     -    0s
*    0     0               0      44.6036941   44.60369  0.00%     -    0s

Cutting planes:
  MIR: 1

Explored 1 nodes (2 simplex iterations) in 0.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: 44.6037 45.1393 117.139 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.460369410857e+01, best bound 4.460369410857e+01, gap 0.0000%
0 -> 0 ship: 45
0 -> 1 ship: 25
1 -> 1 ship: 70
shiftable_bikes[array([[15.64103004]]), array([[-46.41400785]])]
demands[array([[29.35896996]]), array([[141.41400785]])]
inventory[45.0, 95.0]
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thre

Model fingerprint: 0x69fcf86e
Variable types: 4 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Presolve removed 12 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros
Variable types: 3 continuous, 2 integer (0 binary)

Root relaxation: objective 5.966568e-02, 1 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.05967    0    1          -    0.05967      -     -    0s
H    0     0                       0.0946846    0.05967  37.0%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.11 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 0.0946846 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.468460083008e-02, best bound 9.468460082


Explored 1 nodes (2 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: 40.0046 40.9906 58.9906 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000461053821e+01, best bound 4.000461053821e+01, gap 0.0000%
0 -> 0 ship: 56
0 -> 1 ship: 7
1 -> 1 ship: 77
shiftable_bikes[array([[-0.02611679]], dtype=float32), array([[0.02742263]], dtype=float32)]
demands[array([[56.026115]], dtype=float32), array([[83.97258]], dtype=float32)]
inventory[56.0, 84.0]
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 10 columns and 24 nonzeros
Model fingerprint: 0x3313707a
Variable types: 4 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+02]
Presolve removed 12 rows and 5 columns
Presolve time: 0.

...

Thread count was 4 (of 4 available processors)

Solution count 1: 0.115448 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.154479980469e-01, best bound 1.154479980464e-01, gap 0.0000%
0 -> 0 ship: 70
1 -> 1 ship: 70
shiftable_bikes[array([[-0.06230335]], dtype=float32), array([[-0.00985518]], dtype=float32)]
demands[array([[70.0623]], dtype=float32), array([[70.00986]], dtype=float32)]
inventory[70.0, 70.0]
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 10 columns and 24 nonzeros
Model fingerprint: 0x819a2d63
Variable types: 4 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+02]
Presolve removed 12 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros
Variable types: 3 continuous, 2 integer (0 


Optimal solution found (tolerance 1.00e-04)
Best objective 5.567770385742e+01, best bound 5.567770385742e+01, gap 0.0000%
0 -> 0 ship: 70
1 -> 1 ship: 70
shiftable_bikes[array([[-0.05069443]], dtype=float32), array([[0.03597349]], dtype=float32)]
demands[array([[70.0507]], dtype=float32), array([[69.96403]], dtype=float32)]
inventory[70.0, 70.0]
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 10 columns and 24 nonzeros
Model fingerprint: 0x00098a82
Variable types: 4 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+02]
Presolve removed 12 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros
Variable types: 3 continuous, 2 integer (0 binary)

Root relaxation: objective 1.668167e-01, 1 iterations, 0.00 seconds



Root relaxation: objective 7.872009e-02, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.07872    0    1          -    0.07872      -     -    0s
H    0     0                       0.1537018    0.07872  48.8%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 0.153702 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.537017822266e-01, best bound 1.537017822257e-01, gap 0.0000%
0 -> 0 ship: 77
1 -> 1 ship: 63
shiftable_bikes[array([[-0.02497418]], dtype=float32), array([[16.471556]], dtype=float32)]
demands[array([[77.02497]], dtype=float32), array([[46.528442]], dtype=float32)]
inventory[77.0, 63.0]
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a mode

Presolve removed 12 rows and 5 columns
Presolve time: 0.00s
Presolved: 1 rows, 5 columns, 5 nonzeros
Variable types: 3 continuous, 2 integer (0 binary)

Root relaxation: objective 6.615473e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   66.15473    0    1          -   66.15473      -     -    0s
H    0     0                      66.2422333   66.15473  0.13%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.07 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 66.2422 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.624223327637e+01, best bound 6.624223327637e+01, gap 0.0000%
0 -> 0 ship: 70
1 -> 1 ship: 70
shiftable_bikes[array([[-0.02481095]], dtype=float32), array([[0.02872847]], dtype=float32)]
demands[array([[70.02481]], dtype=float32), array([[69.971275]], dtype=float32)]
inventory[70.0, 

,penalty cost,transportation cost,demand distribution,demand parameters,Objective
0,2,"[[0, 1], [1, 0]]",normal,5,10.757783
1,2,"[[0, 1], [1, 0]]",normal,10,13.202528
2,2,"[[0, 1], [1, 0]]",poisson,1,10.964758
3,2,"[[0, 1], [1, 0]]",poisson,2,25.318037
4,2,"[[0, 1], [3, 0]]",normal,5,34.501263
5,2,"[[0, 1], [3, 0]]",normal,10,24.178539
6,2,"[[0, 1], [3, 0]]",poisson,1,18.842306
7,2,"[[0, 1], [3, 0]]",poisson,2,16.187084
8,4,"[[0, 1], [1, 0]]",normal,5,34.219465
9,4,"[[0, 1], [1, 0]]",normal,10,32.826304
